## 1. 필요 라이브러리 호출

In [1]:
import pandas as pd
import requests
import json

path = "/Users/kook/Desktop/"

## 2. 감성분석 api 호출 및 함수 정의

In [2]:
import requests
import json
import pandas as pd

def Sentiment_API(df, column_name):

    client_id = '5ex4s8ep3j'
    client_secret = 'FOPrm2MzgLrbVzVE0I0fs8oKDzwLEms5bZTdfQ6z'
    url = "https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    sentiments = []

    for content in df[column_name]:
        # 950글자까지로 지정
        # API가 1000까지만 처리됨
        data = {
            "content": content[:950]  
        }
        response = requests.post(url, data=json.dumps(data), headers=headers)
        result = json.loads(response.text)

        highlights = []
        for sentence in result["sentences"]:
            highlights.append(sentence.get("highlights", []))

        sentiments.append({
            "label": result["document"]["sentiment"],
            "confidence": result["document"]["confidence"],
            "highlights": highlights
        })

    sentiments_df = pd.DataFrame(sentiments)
    df = pd.concat([df, sentiments_df], axis=1)
    
    return df

def extract_highlighted_text(df, title_column, highlights_column):
    highlighted_texts = []

    for idx, row in df.iterrows():
        title = row[title_column]
        highlights = row[highlights_column]
        extracted_texts = []

        for highlight_set in highlights:
            for highlight in highlight_set:
                offset = highlight['offset']
                length = highlight['length']
                extracted_texts.append(title[offset:offset + length])
        
        highlighted_texts.append(' '.join(extracted_texts))
    
    df['highlighted_texts'] = highlighted_texts
    return df

In [3]:
# 감성분석 데이터셋
df = pd.read_excel("금산천벚꽃축제 네이버뉴스 크롤링.xlsx")
df

,Unnamed: 0,title,href
0,0,관광 최전방인데 바가지요금 '낙인'…살 길은?,https://www.news1.kr/articles/5437790
1,1,금산 벚꽃축제 일주일 연기,http://www.ccnnews.co.kr/news/articleView.html...
2,2,'진해군항제' 보다 여기?··· 떠오른 벚꽃 축제 장소,http://www.enewstoday.co.kr/news/articleView.h...
3,3,'끝나지 않은 팡파르'…충남권서 벚꽃축제 이어 다채로운 행사 개최,http://www.chungnamilbo.co.kr/news/articleView...
4,4,"1위 여의도 벚꽃축제·2위 진해군항제·3위 안동벚꽃축제, 4월 2주차 랭키파이...",https://cwn.kr/article/1065571509920448
...,...,...,...
95,95,벚꽃만발 경남 봄 축제장으로 오세요,http://www.gndomin.com/news/articleView.html?i...
96,96,"공군 교육사령부, 30~31일 벚꽃축제 부대 개방",http://www.asiatoday.co.kr/view.php?key=201903...
97,97,"공군 교육사령부,'가족과 함께 사색하는 벚꽃축제'개최",http://www.newsfreezone.co.kr/news/articleView...
98,98,아직 끝나지 않은 2018 벚꽃 축제 일정 어디? 명소 10곳,http://www.sporbiz.co.kr/news/articleView.html...


## 3. 감성분석

In [4]:
df_sentiment = Sentiment_API(df, 'title')

In [5]:
df_with_highlights = extract_highlighted_text(df_sentiment, 'title', 'highlights')

In [6]:
df_sentiment

,Unnamed: 0,title,href,label,confidence,highlights,highlighted_texts
0,0,관광 최전방인데 바가지요금 '낙인'…살 길은?,https://www.news1.kr/articles/5437790,negative,"{'negative': 99.98196, 'positive': 0.007564219...","[[{'offset': 16, 'length': 2}]]",낙인
1,1,금산 벚꽃축제 일주일 연기,http://www.ccnnews.co.kr/news/articleView.html...,neutral,"{'negative': 0.09631314, 'positive': 0.0224453...","[[{'offset': 0, 'length': 14}]]",금산 벚꽃축제 일주일 연기
2,2,'진해군항제' 보다 여기?··· 떠오른 벚꽃 축제 장소,http://www.enewstoday.co.kr/news/articleView.h...,neutral,"{'negative': 0.043967977, 'positive': 0.252163...","[[{'offset': 18, 'length': 12}]]",떠오른 벚꽃 축제 장소
3,3,'끝나지 않은 팡파르'…충남권서 벚꽃축제 이어 다채로운 행사 개최,http://www.chungnamilbo.co.kr/news/articleView...,neutral,"{'negative': 0.045031976, 'positive': 1.022919...","[[{'offset': 13, 'length': 23}]]",충남권서 벚꽃축제 이어 다채로운 행사 개최
4,4,"1위 여의도 벚꽃축제·2위 진해군항제·3위 안동벚꽃축제, 4월 2주차 랭키파이...",https://cwn.kr/article/1065571509920448,neutral,"{'negative': 0.024277488, 'positive': 0.043339...","[[{'offset': 32, 'length': 11}]]",4월 2주차 랭키파이
...,...,...,...,...,...,...,...
95,95,벚꽃만발 경남 봄 축제장으로 오세요,http://www.gndomin.com/news/articleView.html?i...,neutral,"{'negative': 0.059205964, 'positive': 0.991595...","[[{'offset': 0, 'length': 19}]]",벚꽃만발 경남 봄 축제장으로 오세요
96,96,"공군 교육사령부, 30~31일 벚꽃축제 부대 개방",http://www.asiatoday.co.kr/view.php?key=201903...,neutral,"{'negative': 0.15120982, 'positive': 0.3330488...","[[{'offset': 10, 'length': 17}]]",30~31일 벚꽃축제 부대 개방
97,97,"공군 교육사령부,'가족과 함께 사색하는 벚꽃축제'개최",http://www.newsfreezone.co.kr/news/articleView...,neutral,"{'negative': 0.05601185, 'positive': 0.4311116...","[[{'offset': 10, 'length': 16}]]",가족과 함께 사색하는 벚꽃축제
98,98,아직 끝나지 않은 2018 벚꽃 축제 일정 어디? 명소 10곳,http://www.sporbiz.co.kr/news/articleView.html...,neutral,"{'negative': 0.09574045, 'positive': 0.0446443...","[[{'offset': 0, 'length': 26}], [{'offset': 1,...",아직 끝나지 않은 2018 벚꽃 축제 일정 어디 직 끝나지


## 4. 감성분석 기여단어 추출(TF-IDF vectorizer 활용)

In [7]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

# 형태소 분석기 초기화
okt = Okt()

# highlighted_texts 컬럼을 형태소 단위로 나누기
df_sentiment['tokenized_highlighted_texts'] = df_sentiment['highlighted_texts'].apply(lambda x: ' '.join(okt.nouns(x)))

# TF-IDF 벡터라이저 초기화
vectorizer = TfidfVectorizer()

# TF-IDF 행렬 생성
tfidf_matrix = vectorizer.fit_transform(df_sentiment['tokenized_highlighted_texts'])

# 단어 리스트 추출
terms = vectorizer.get_feature_names_out()

# 제목별로 중요한 단어 1개 추출
def extract_top_word(row):
    tfidf_vector = tfidf_matrix[row.name].toarray().flatten()
    top_index = tfidf_vector.argmax()
    top_word = terms[top_index]
    return top_word

# 각 제목별로 중요한 단어 1개 추출하여 'top_word' 컬럼에 저장
df_sentiment['top_word'] = df_sentiment.apply(extract_top_word, axis=1)

# 필요한 컬럼만 선택하여 최종 데이터프레임 생성
df_final = df_sentiment[['highlighted_texts', 'top_word', 'label', 'confidence']]

# 최종 데이터프레임 출력
print(df_final)

                    highlighted_texts top_word     label  \
0                                  낙인       낙인  negative   
1                      금산 벚꽃축제 일주일 연기       연기   neutral   
2                        떠오른 벚꽃 축제 장소       장소   neutral   
3             충남권서 벚꽃축제 이어 다채로운 행사 개최       행사   neutral   
4                         4월 2주차 랭키파이       키파   neutral   
..                                ...      ...       ...   
95                벚꽃만발 경남 봄 축제장으로 오세요       경남   neutral   
96                  30~31일 벚꽃축제 부대 개방       개방   neutral   
97                   가족과 함께 사색하는 벚꽃축제       사색   neutral   
98  아직 끝나지 않은 2018 벚꽃 축제 일정 어디 직 끝나지        일정   neutral   
99     벚꽃길 걸으며 봄꽃 여행을  꽃길 걸으며 봄꽃 여행을        여행   neutral   

                                           confidence  
0   {'negative': 99.98196, 'positive': 0.007564219...  
1   {'negative': 0.09631314, 'positive': 0.0224453...  
2   {'negative': 0.043967977, 'positive': 0.252163...  
3   {'negative': 0.045031976, 'positive': 1.022919...  

In [9]:
df_final.to_excel('금산천벚꽃축제 감성분석 최고 기여 단어.xlsx', index=False)